In [1]:
import json
import requests
import pandas as pd
import time
import datetime

In [2]:
lol_api_key = 'RGAPI-9badd027-7154-41b1-8143-404fb589ae23'

In [3]:
URLS = {'challenge' : 'https://kr.api.riotgames.com/lol/league/v3/challengerleagues/by-queue/RANKED_SOLO_5x5?api_key=',
        'master' : 'https://kr.api.riotgames.com/lol/league/v3/masterleagues/by-queue/RANKED_SOLO_5x5?api_key=',
        'summoners' : 'https://kr.api.riotgames.com/lol/summoner/v3/summoners/',
        'matches' : 'https://kr.api.riotgames.com/lol/match/v3/matchlists/by-account/'
       }

In [4]:
# challenge league 선수들 정보 불러옴
url = requests.get(URLS['challenge']+lol_api_key)
print(url.status_code)
challenge = url.json()

200


In [5]:
# master league 선수들 정보 불러옴
url = requests.get(URLS['master']+lol_api_key)
print(url.status_code)
master = url.json()

200


In [6]:
# match를 모을 target을 challenge, master 리그 summerner들로 잡음
df_target_origin = pd.DataFrame(challenge['entries']).append(pd.DataFrame(master['entries']), ignore_index=True)
df_target_origin[df_target_origin.playerOrTeamName == 'Hide on bush']

,freshBlood,hotStreak,inactive,leaguePoints,losses,playerOrTeamId,playerOrTeamName,rank,veteran,wins
76,False,True,False,1215,795,4460427,Hide on bush,I,True,892


In [ ]:
# X-App-Rate-Limit에 맞게 시간을 쉬면서 돌려줌 status_code=429 는 api호출 한계 초과임
summoners=[]
counts=0
for i in df_target.playerOrTeamId:
    time.sleep(0.84)
#     counts += 1
#     print(counts)
    url = requests.get(URLS['summoners'] + i + '?api_key=' +lol_api_key)
    while url.status_code == 429:
        time.sleep(1)
        url = requests.get(URLS['summoners'] + i + '?api_key=' + lol_api_key)
    if url.status_code != 200:
        print(i, url.status_code)
    else:
        summoners.append(url.json())

pd.DataFrame(summoners).to_csv('summonors.csv', index = False)

In [7]:
df_summonors = pd.read_csv('summonors.csv')[['accountId','id']]

In [8]:
# 이렇게 안하면 merge시 조인이 안됨 <- 데이터 타입이 달라서 생기는 문제인데 꼭 이렇게 해야 할까?
df_target_origin['playerOrTeamId'] = df_target_origin['playerOrTeamId'].astype(int)

In [9]:
df_target = pd.merge(df_target_origin, df_summonors, left_on='playerOrTeamId', right_on = 'id', 
                     how='inner')

In [10]:
df_target[df_target.playerOrTeamName == 'Hide on bush']

,freshBlood,hotStreak,inactive,leaguePoints,losses,playerOrTeamId,playerOrTeamName,rank,veteran,wins,accountId,id
76,False,True,False,1215,795,4460427,Hide on bush,I,True,892,3440481,4460427


# match

In [11]:
# version별 분석을 하고 싶어서 기준일을 timestamp로 지정한다.
ver7_19_timestamp_begin = int(datetime.datetime.timestamp(datetime.datetime(2017,9,28,0,0,0))) * 1000
ver7_19_timestamp_end = int(datetime.datetime.timestamp(datetime.datetime(2017,10,4,0,0,0))) * 1000

print(ver7_19_timestamp_begin, ver7_19_timestamp_end)

1506524400000 1507042800000


In [41]:
# match 를 선수별로 모은다.
# 선수당 max 100개만 모은다. 라이엇 정책이 1회 조회에 100개. / 100개 이상으로도 돌릴 수 있을 것 같긴 한데 이번 것에선 여기까지.
matches = dict()
exeptions = []
counts = 0
for i in df_target.accountId:
    time.sleep(0.84)
#     counts += 1
#     print(counts, i)
    url = requests.get(URLS['matches'] + str(i) + '?endTime=' + str(ver7_19_timestamp_end) + '&beginTime=' + \
                       str(ver7_19_timestamp_begin) + '&api_key=' + lol_api_key)
    while url.status_code == 429:
        time.sleep(1)
        url = requests.get(URLS['matches'] + str(i) + '?endTime=' + str(ver7_19_timestamp_end) + '&beginTime=' + \
                           str(ver7_19_timestamp_begin) + '&api_key=' + lol_api_key)
    if url.status_code != 200:
        print(i, url.status_code)
        exeptions.append([i, url.status_code])
    else:
        matches[str(i)] = url.json()

with open('matches.json', 'w') as js:
     json.dump(matches, js)

209455940 404
209468211 404
209455696 404
7751954 404
209430208 404
204544908 404
204522812 404
208351516 404
548226 404
1138752 404
1490224 404
209455774 404
209455773 404
208971958 404
1769449 404
7868833 404
2265030 404
203024162 404
209651402 404
203415016 404
7530974 404
209213517 404
208057838 404
204672058 404
209034235 404
3547852 404
207570975 404
8036131 404
207938018 404
2500731 404
2212413 404
209455851 404
200734451 404
208756914 404
4170045 404
841749 404
3860080 404
209333398 404
201461099 404
201533956 404
2619086 404
722871 404
3443530 404
206720203 404


In [42]:
with open('matches.json', 'r') as js:
     mataches_json = json.load(js)

In [44]:
# json.dumps(matches)

In [58]:
mataches_json['2533789'].keys()

dict_keys(['totalGames', 'endIndex', 'matches', 'startIndex'])

In [81]:
for key in mataches_json:
    ㅗㅜprint(key)
    print(values)

2533789
205964102
1110666
208670326
7859270
3212081
209084978
1526872
3145436
1309737
266599
206737570
202047579
208904782
200933520
4818546
207987339
209376444
204577481
208177961
1646013
209132696
207609618
3531497
2474930
201843256
209449434
6255422
208866397
2204527
203744805
209197502
205825868
202561023
205480506
202620815
206305916
207990919
205036558
5592043
200129667
3380468
205146069
207493015
3101966
209619770
208631140
5119391
208752912
1892370
1302978
1367961
203921694
6341504
669822
4876431
200575439
208561577
204531622
200060017
207677337
209682432
205937251
208236402
6477601
201481153
1198324
205167316
208719067
2881442
209712481
203294131
207692942
5548984
3494294
203870735
1640424
205923395
4094465
204239071
794227
204196076
4930311
202583967
209078773
251709
208757003
5046059
208732190
208543788
203509833
209708070
3946176
2099284
7850231
209274483
206560208
200555547
208752764
5748929
209254012
202998809
6924001
209429976
927790
207984405
3398806
5797434
208746217
5

In [69]:
gameIds = []
for ids in mataches_json
for i in range(mataches_json['2533789']['endIndex']):
    gameIds.append(mataches_json['2533789']['matches'][i]['gameId'])

In [70]:
gameIds

[2971071986,
 2970997025,
 2970972521,
 2970933850,
 2970789670,
 2970371522,
 2970317517,
 2970332585,
 2970225168,
 2970077071,
 2970001189,
 2969832780,
 2969055087,
 2969080996,
 2968866856,
 2968842624,
 2968696878,
 2968711010,
 2968654928,
 2968614170,
 2968506562,
 2967508212,
 2967533602,
 2967478656,
 2967452096,
 2967405609,
 2967400209,
 2966296203,
 2966228855,
 2966212405,
 2966185648,
 2966099197,
 2966112953,
 2966049698,
 2966063779,
 2964400447,
 2964298505,
 2964315340]

# 참고자료

In [ ]:
API_KEY = os.environ['LOL_API_KEY']
URLS = {
        'NAME_TO_ID' : 'https://na1.api.riotgames.com/lol/summoner/v3/summoners/by-name/{name}',
        'ID_TO_NAME' : 'https://na1.api.riotgames.com/lol/summoner/v3/summoners/by-account/{id}',
        'MATCHES_BY_ID' : 'https://na1.api.riotgames.com/lol/match/v3/matchlists/by-account/{id}?season=8',
        'RECENT_MATCHES_BY_ID' : 'https://na1.api.riotgames.com/lol/match/v3/matchlists/by-account/{id}/recent',
        'MATCH_INFO': 'https://na1.api.riotgames.com/lol/match/v3/matches/{id}', 
        'RECENT_TEN' : 'https://na.api.riotgames.com/api/lol/NA/v1.3/game/by-summoner/{summonerId}/recent',
        }
HEADERS = {'X-Riot-Token' : API_KEY}

CHAMP_BY_ID = {}
with open('data/champ_by_id.json') as f:
    CHAMP_BY_ID = json.load(f)

CHAMP_BY_NAME = {}
with open('data/champ_by_name.json') as f:
    CHAMP_BY_NAME = json.load(f)


print(CHAMP_BY_ID['data']['236'])

In [ ]:
from apiKeys import API_KEY
import requests
import sys
import pprint

USERNAME = sys.argv[1]
BASE_URL = "https://na1.api.riotgames.com/lol/"
SUMMONER_ID_URL = BASE_URL + "summoner/v3/summoners/by-name/"
MASTERY_URL = BASE_URL + "platform/v3/masteries/by-summoner/"

r = requests.get(SUMMONER_ID_URL + USERNAME + "?api_key=" + API_KEY)
print r.status_code
data = r.json()
print data
print data["name"]
print data["id"]
summoner_id = data["id"]
r = requests.get(MASTERY_URL + str(summoner_id) + "?api_key=" + API_KEY)
pprint.pprint(r.json())
